## Webscrapper

### I. Introduction

The purpose of this notebook is to build a webscrapper that will extract transcript and metadata from the following website: https://www.podgist.com/joe-rogan-experience/index.html  
This webiste compiles the unofficial transcript of the Joe Rogan Experience podcast since its inception

### II. Webscrapping Metadata
We import the following metadata in a first dataframe:  
> URL  
> Title of the podcast  
> Duration  
> Date  


In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint

In [2]:
url='https://www.podgist.com/joe-rogan-experience/index.html'
result = requests.get(url)

def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)
soup = BeautifulSoup(result.text, 'html.parser')

getAndParseURL(url)

titles = []
duration = []
date = []
text = []
url_all = []

for link in soup.findAll('a'):
        a='https://www.podgist.com'+link.get('href')
        url_all.append(a)

for i in soup.findAll("td", class_ = "duration"):
        du=(i.contents) 
        duration.append(du)

for j in soup.findAll("td", class_ = "date"):
        da=(j.contents[0]) 
        date.append(da)

for k in soup.findAll("td", class_ = "title"):
        ti=(k.contents[0]) 
        titles.append(ti)


In [3]:
#Create the dataframe
df = pd.DataFrame(list(zip(date, titles, duration)), 
               columns =['date', 'title', 'duration']) 
df = df.drop(index=1097)
# remove #480 - Duncan Trussell & DJ Douggpound 	2:58:53 	2014-04-02 from the list
df.shape

(1655, 3)

### III. Scrapping the transcript

In [5]:
#Let's scrap the actual transcript from each URL. 
#It takes quite some time to load, and we added the sleep function to avoid spamming the website

text = []

#start at #2 since the first urls are redirecting to the podcast website and remove #480 - Duncan Trussell & DJ Douggpound 	2:58:53 	2014-04-02 from the list

for link in url_all[2:1099]:
    
    para = []
    soup1 = BeautifulSoup(link, 'html.parser')
    result1 = requests.get(soup1)
    soup1 = BeautifulSoup(result1.text, 'html.parser')
   
    for match in soup1.findAll('span'):
                match.unwrap()

    for s in soup1.findAll('div',{'class':'transcription'}):
            t=(s.contents[1])
            para.append(t)
        
    text.append(para)
        
    sleep(randint(1,2))#put the sleep function to avoid spamming the website


C:\Users\benja\anaconda3\lib\site-packages\bs4\__init__.py:389: UserWarning: "https://www.podgist.com/joe-rogan-experience/800-bobcat-goldthwait/index.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\benja\anaconda3\lib\site-packages\bs4\__init__.py:389: UserWarning: "https://www.podgist.com/joe-rogan-experience/798-alison-rosen/index.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\benja\anaconda3\lib\site-packages\bs4\__init__.py:389: UserWarning: "https://www.podgist.com/joe-rogan-experience/797-alex-allyson-grey/index.html" looks like a URL. Beautiful Soup is not an HTTP cl

### IV. Organising in dataframe and exporting to CSV

In [6]:
df2 = pd.DataFrame(list(zip(text)), 
               columns =['text']) 
df2

,text
0,"[Hey friends, how you doing this episode, the ..."
1,"[Hello, friends, welcome to the show this epis..."
2,"[Pillow friends, welcome to the show this epis..."
3,"[Hello, friends, welcome to the show this epis..."
4,"[Oh hi, friends welcome to the show this absur..."
...,...
1092,[He keeps getting. Burma is hit this. Why? I t...
1093,"[Baby Jesus POGO stick , easy fashion square, ..."
1094,"[Boom, and here we are, ladies and gentlemen, ..."
1095,"[Hey everybody here, this , thing is a no bull..."


In [49]:
df1 = pd.DataFrame(list(zip(date, titles, duration, text)), 
               columns =['date', 'title', 'duration', 'text']) 
df1

,date,title,duration,text
0,2020-06-03,#1485 - Krystal & Saagar,[],"[Hey friends, how you doing this episode, the ..."
1,2020-06-02,[#1484 - Reggie Watts],[2:46:32],"[Hello, friends, welcome to the show this epis..."
2,2020-05-29,[#1483 - Jesus Trejo],[2:52:47],"[Pillow friends, welcome to the show this epis..."
3,2020-05-28,[#1482 - Jordan Jonas],[2:22:39],"[Hello, friends, welcome to the show this epis..."
4,2020-05-27,[JRE MMA Show #96 with Justin Gaethje & Trevor...,[2:58:39],"[Oh hi, friends welcome to the show this absur..."
...,...,...,...,...
1092,2014-09-15,[#549 - Big Jay Oakerson],[2:55:24],[He keeps getting. Burma is hit this. Why? I t...
1093,2014-09-11,[#548 - Tim Burnett],[2:58:43],"[Baby Jesus POGO stick , easy fashion square, ..."
1094,2014-09-10,[#547 - Joe DeRosa],[2:50:28],"[Boom, and here we are, ladies and gentlemen, ..."
1095,2014-09-09,#546 - Mike Dolce,[],"[Hey everybody here, this , thing is a no bull..."


In [50]:
df.shape

(1655, 3)

In [51]:
df1.shape

(1097, 4)

In [52]:
df2.shape

(1097, 1)

In [54]:
df1.to_csv('jrecontent.csv')
df.to_csv('jredata.csv')